[Problem 1] Looking back on scratch

In [2]:
"""
TensorFlow Task
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()


dataset_path ="import/iris.csv"
df = pd.read_csv(dataset_path)


df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int32)[:, np.newaxis]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

[Problem 2] Consider the correspondence between scratch and TensorFlow

In [3]:
#get mini batch 
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int32)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


#hyperparameters 
learning_rate = 0.001
batch_size = 8
num_epochs = 100

n_hidden1 = 64
n_hidden2 = 128
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# tensorflow array 
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

## dataloader 
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# model architecture
def example_net(x):

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# model with initialized parameters                       
logits = example_net(X)


#loss function 
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

## accuracy 
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

print("Not trained model's accuracy:", accuracy)


### initialize training process
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


Not trained model's accuracy: Tensor("Mean_1:0", shape=(), dtype=float32)
Epoch 0, loss : 10.6622, val_loss : 7.9836, acc : 0.500, val_acc : 0.438
Epoch 1, loss : 10.6124, val_loss : 4.6958, acc : 0.625, val_acc : 0.750
Epoch 2, loss : 6.3197, val_loss : 5.0364, acc : 0.500, val_acc : 0.500
Epoch 3, loss : 5.6791, val_loss : 2.8361, acc : 0.625, val_acc : 0.688
Epoch 4, loss : 2.6021, val_loss : 1.7236, acc : 0.625, val_acc : 0.688
Epoch 5, loss : 1.0454, val_loss : 0.8632, acc : 0.750, val_acc : 0.812
Epoch 6, loss : 0.1228, val_loss : 0.4517, acc : 0.875, val_acc : 0.938
Epoch 7, loss : 0.0506, val_loss : 0.1325, acc : 1.000, val_acc : 0.938
Epoch 8, loss : 0.0055, val_loss : 0.0653, acc : 1.000, val_acc : 0.938
Epoch 9, loss : 0.0001, val_loss : 0.2974, acc : 1.000, val_acc : 0.875
Epoch 10, loss : 0.0164, val_loss : 0.0076, acc : 1.000, val_acc : 1.000
Epoch 11, loss : 0.0001, val_loss : 0.4042, acc : 1.000, val_acc : 0.875
Epoch 12, loss : 0.0571, val_loss : 0.0259, acc : 1.000, 

[Problem 3] Create a model of Iris using all three types of objective variables

In [4]:
from sklearn.preprocessing import OneHotEncoder

dataset_path ="import/iris.csv"
df = pd.read_csv(dataset_path)

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2

enc = OneHotEncoder(handle_unknown='ignore')
y = enc.fit_transform(y[:,np.newaxis]).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [5]:
y_train.shape

(96, 3)

In [6]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int32)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [7]:
learning_rate = 0.01
batch_size = 8
num_epochs = 100

n_hidden1 = 64
n_hidden2 = 128
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

#model initiliaze                            
logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))

#not trained model's accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## training 
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 117.8301, val_loss : 132.1389, acc : 0.250, val_acc : 0.375
Epoch 1, loss : 30.5931, val_loss : 23.6254, acc : 0.500, val_acc : 0.708
Epoch 2, loss : 8.4454, val_loss : 14.6455, acc : 0.375, val_acc : 0.708
Epoch 3, loss : 5.8496, val_loss : 14.9514, acc : 0.500, val_acc : 0.708
Epoch 4, loss : 1.0529, val_loss : 4.4697, acc : 0.875, val_acc : 0.833
Epoch 5, loss : 3.0873, val_loss : 2.1987, acc : 0.875, val_acc : 0.833
Epoch 6, loss : 0.0876, val_loss : 2.6880, acc : 0.875, val_acc : 0.875
Epoch 7, loss : 0.0017, val_loss : 2.0899, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 3.1497, val_loss : 5.3175, acc : 0.875, val_acc : 0.750
Epoch 9, loss : 3.4983, val_loss : 8.3434, acc : 0.875, val_acc : 0.708
Epoch 10, loss : 1.8366, val_loss : 6.5514, acc : 0.875, val_acc : 0.750
Epoch 11, loss : 0.0000, val_loss : 1.2962, acc : 1.000, val_acc : 0.875
Epoch 12, loss : 0.0000, val_loss : 1.8105, acc : 1.000, val_acc : 0.917
Epoch 13, loss : 1.3296, val_loss : 10.8819, acc : 0.

[Problem 4] Create a model of House Prices

In [8]:
import pandas as pd
import numpy as np
data_ori = pd.read_csv("import/train.csv")

In [9]:
max_num_of_row = len(data_ori)
data_loss_rate = data_ori.isnull().sum() / max_num_of_row * 100

In [10]:
data_drop_5nan_col = data_ori.dropna(axis=1, thresh=max_num_of_row-5)
print("before:{}".format(len(data_ori.columns.values)))
print("after:{}".format(len(data_drop_5nan_col.columns.values)))

before:81
after:63


In [11]:
data_to_use = data_drop_5nan_col.dropna(axis=0)
data_to_use = data_to_use.reset_index(drop=True)

In [12]:
col_featuer = ['OverallQual',
               'GrLivArea',
               'GarageCars',
               'GarageArea',
               'TotalBsmtSF',
               '1stFlrSF',
               'FullBath',
               'TotRmsAbvGrd',
               'YearBuilt',
               'YearRemodAdd',
              ]

col_target = ['SalePrice']

cols = col_featuer + col_target

In [13]:
data = data_to_use[cols]

In [14]:
import gc 
gc.enable()
gc.collect()

736

In [15]:
del data_ori, data_drop_5nan_col, data_to_use
gc.collect()

0

In [16]:
X = data[col_featuer].values
Y = data[col_target].values

In [17]:
# data split to train, validation and test 
# train for model training 
# validation for model selection
# test for model evaluation 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [18]:
# hyperparameters
learning_rate = 0.01
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 1

# tf tensors
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# dataloader 
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

# model 
def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# not trained model's prediction                             
pred = example_net(X)

# loss for regression 
# mean absolute loss is good for outliers 
loss_op = tf.reduce_mean(tf.abs(Y - pred))

# optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# evaluation metric
r_square = 1 - (tf.reduce_sum(tf.square(Y - pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

#training 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_r_square = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, r_2 = sess.run([loss_op, r_square], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_r_square += r_2
        total_loss /= n_samples
        total_r_square /= n_samples
        val_loss, val_r_square = sess.run([loss_op, r_square], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, r_square : {:.3f}, val_r_square : {:.3f}".format(epoch, loss, val_loss, r_2, val_r_square))
    test_r_square= sess.run(r_square, feed_dict={X: x_test, Y: y_test})
    print("test_r_square : {:.3f}".format(test_r_square))

Epoch 0, loss : 51168.6758, val_loss : 48663.4492, r_square : 0.834, val_r_square : 0.350
Epoch 1, loss : 46546.1719, val_loss : 36217.5586, r_square : 0.855, val_r_square : 0.506
Epoch 2, loss : 43561.5273, val_loss : 35166.3359, r_square : 0.876, val_r_square : 0.505
Epoch 3, loss : 39132.0820, val_loss : 34971.7148, r_square : 0.910, val_r_square : 0.494
Epoch 4, loss : 35614.3320, val_loss : 34364.4609, r_square : 0.929, val_r_square : 0.481
Epoch 5, loss : 33606.6758, val_loss : 33270.1445, r_square : 0.937, val_r_square : 0.490
Epoch 6, loss : 28408.5098, val_loss : 35668.4258, r_square : 0.960, val_r_square : 0.437
Epoch 7, loss : 30865.8184, val_loss : 33182.7148, r_square : 0.953, val_r_square : 0.468
Epoch 8, loss : 23750.2871, val_loss : 34032.2109, r_square : 0.973, val_r_square : 0.432
Epoch 9, loss : 22108.5723, val_loss : 35302.1914, r_square : 0.976, val_r_square : 0.414
Epoch 10, loss : 24735.2090, val_loss : 33761.7500, r_square : 0.971, val_r_square : 0.427
Epoch 11,

[Problem 5] Create a model of MNIST

In [26]:
from keras.datasets import mnist

# MNIST dataset 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255
enc = OneHotEncoder(handle_unknown='ignore')
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis]).toarray()
y_test_one_hot = enc.transform(y_test[:, np.newaxis]).toarray()

In [27]:
#train , val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_one_hot, test_size=0.4, shuffle=True, random_state=42)
print(x_train.shape) 
print(x_val.shape) 

(36000, 784)
(24000, 784)


In [28]:
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 10

#tf tensor
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

#dataloader
get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

# model architecture
def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output

#model 
logits = example_net(X)

# loss 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

#optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#not trained model's result
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
#evaluation metric 
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#model training 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int32)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.5951, val_loss : 1.0959, acc : 0.800, val_acc : 0.678
Epoch 1, loss : 0.4099, val_loss : 0.9002, acc : 0.900, val_acc : 0.771
Epoch 2, loss : 0.3654, val_loss : 0.7361, acc : 0.900, val_acc : 0.825
Epoch 3, loss : 0.2743, val_loss : 0.5697, acc : 0.900, val_acc : 0.862
Epoch 4, loss : 0.0578, val_loss : 0.4629, acc : 1.000, val_acc : 0.899
Epoch 5, loss : 0.0412, val_loss : 0.3800, acc : 1.000, val_acc : 0.909
Epoch 6, loss : 0.0339, val_loss : 0.3839, acc : 1.000, val_acc : 0.907
Epoch 7, loss : 0.0413, val_loss : 0.3896, acc : 1.000, val_acc : 0.911
Epoch 8, loss : 0.0184, val_loss : 0.3581, acc : 1.000, val_acc : 0.919
Epoch 9, loss : 0.0226, val_loss : 0.3439, acc : 1.000, val_acc : 0.921
test_acc : 0.925
